In [ ]:
from __future__ import print_function
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
from keras.models import model_from_json

Parameters

In [ ]:
maxlen = 40
step = 1
batch_size=128
seed="le gouvernement"
nb_units=128
epochs=2
nb_char=140
lr_rate=0.001

Input file

In [ ]:
Input fileimport codecs
input_file='data_blog'
training_file = input_file+'.txt'
text = codecs.open(training_file, 'r',"utf-8").read()

Creating vocabulary

In [ ]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

Probability

In [ ]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

Reading network parameters from json file

In [ ]:

print('Build model...')
model = Sequential()
model.add(LSTM(nb_units, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))
Input file
optimizer = RMSprop(lr=lr_rate)
model_json = model.to_json()
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
# load weights into new model
model.load_weights("model.h5")
print("Loaded model from disk")
model.compile(loss='categorical_crossentropy', optimizer=optimizer)


Generating the tweets

In [ ]:
start_index = random.randint(0,batch_size - maxlen - 1)

for diversity in [0.2, 0.5, 1.0, 1.2]:
    print()
    print('----- diversity:', diversity)

    generated = ''
    #sentence = text[start_index: start_index + maxlen]
    sentence=seed
    generated += sentence
    print('----- Generating with seed: "' + sentence + '"')
    print('\nGenerated')
    sys.stdout.write(generated)

    for i in range(nb_char):
        x = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x[0, t, char_indices[char]] = 1.

        preds = model.predict(x, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()

Publishing the tweet using tweepy

In [ ]:
import tweepy
from tweepy import OAuthHandler
 
consumer_key = ''
consumer_secret = ''
access_token = ''
access_secret = ''
 
auth = OAuthHandler(consumer_key, consumer_secret)Input file
auth.set_access_token(access_token, access_secret)
 
api = tweepy.API(auth)
tweet=api.update_status(sentence)